# Question 2.1 

In [1]:
import pandas as pd 
df = pd.read_csv("files/Phase2.csv") 

### Description  des donner  

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25192 entries, 0 to 25191
Data columns (total 42 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     25192 non-null  int64  
 1   protocol_type                25192 non-null  object 
 2   service                      25192 non-null  object 
 3   flag                         25192 non-null  object 
 4   src_bytes                    25192 non-null  int64  
 5   dst_bytes                    25192 non-null  int64  
 6   land                         25192 non-null  int64  
 7   wrong_fragment               25192 non-null  int64  
 8   urgent                       25192 non-null  int64  
 9   hot                          25192 non-null  int64  
 10  num_failed_logins            25192 non-null  int64  
 11  logged_in                    25192 non-null  int64  
 12  num_compromised              25192 non-null  int64  
 13  root_shell      

In [3]:
print(df.describe())

           duration     src_bytes     dst_bytes          land  wrong_fragment  \
count  25192.000000  2.519200e+04  2.519200e+04  25192.000000    25192.000000   
mean     305.054104  2.433063e+04  3.491847e+03      0.000079        0.023738   
std     2686.555640  2.410805e+06  8.883072e+04      0.008910        0.260221   
min        0.000000  0.000000e+00  0.000000e+00      0.000000        0.000000   
25%        0.000000  0.000000e+00  0.000000e+00      0.000000        0.000000   
50%        0.000000  4.400000e+01  0.000000e+00      0.000000        0.000000   
75%        0.000000  2.790000e+02  5.302500e+02      0.000000        0.000000   
max    42862.000000  3.817091e+08  5.151385e+06      1.000000        3.000000   

            urgent           hot  num_failed_logins     logged_in  \
count  25192.00000  25192.000000       25192.000000  25192.000000   
mean       0.00004      0.198039           0.001191      0.394768   
std        0.00630      2.154202           0.045418      0.4888

### Gestion des valeur null

In [4]:
print(' affichage des valeur null par column : ', df.isnull().sum()) #afficher lles valeur null par column 
df = df.fillna(df.median())# remplire les valeur nulle par la valeur median , nous navont pas de valuer null pour notre cas 

 affichage des valeur null par column :  duration                       0
protocol_type                  0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed_logins              0
logged_in                      0
num_compromised                0
root_shell                     0
su_attempted                   0
num_root                       0
num_file_creations             0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
is_guest_login                 0
count                          0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate                  0
di

### Gestion des valeurs redondantes

In [5]:
df= df.drop_duplicates(keep='first').reset_index()# dellting of all the rows that are duplicate 

### Transformation des donner 
utilisation dune codification one hoting 

In [6]:
txtDataColumn = ['protocol_type','service','flag']
df = pd.get_dummies(df,columns=txtDataColumn) 

### les info les plus pertinente a garder 

In [7]:
t = df["class"].value_counts()
print("pourcentage des noraml :",round( t["normal"]*100/df["class"].count(), 3),"%")
print("pourcentage des anomaly :",round(t["anomaly"]*100/df["class"].count(), 3),"%")

pourcentage des noraml : 53.386 %
pourcentage des anomaly : 46.614 %


### Réponse à la deuxième partie de la question : 
#### les changement ajouter au donner sont l’ajout de nouvelle donner de la class anomaly, ce qui a  provoquer l’augmentation du nombre de donner dans notre data-set ainsi que la création d’un équilibre entre le nombre de donner des deux class. avec un pourcentage de 53.386 % pour les normal, et 46.614 % pour les anomaly


# Question 2.2

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
x = df.drop("class",axis = 1) #getting the features of the data in a matrix 
y = df["class"] #getting the labes of the data in a vector 
X_train, X_test , y_train , y_test = train_test_split(x , y, test_size=0.3,stratify=y ) 

param_grid= {'n_neighbors' : [3,5,7,9,11,13,15,17], 
'metric' : ['manhattan', 'euclidean']
}
grid = GridSearchCV(KNeighborsClassifier(),param_grid, cv = 5) #initiamlisation du grid estimator 
grid.fit(X_train, y_train)# entrainement du grid (kfold)
print('les meilleur parametre pour notre knn sont : ', grid.best_params_)
bestEstimator = grid.best_estimator_#on garde le modelle avec les mielleur parametre 
y_pred=bestEstimator.predict(X_test) 

print('\npour la class ANOMALY :')
af1score = f1_score(y_test, y_pred, pos_label = 'anomaly')
print('precision = ',round(precision_score(y_test,y_pred, pos_label = 'anomaly'),3))
print('recall = ',round(recall_score(y_test,y_pred,pos_label = 'anomaly'),3))
print('f1_score = ',round(af1score,3))

print("\n")


print('pour la class NORMAL :')
nf1score = f1_score(y_test, y_pred, pos_label = 'normal')
print('precision=',round(precision_score(y_test,y_pred, pos_label = 'normal'),3))
print('recall=',round(recall_score(y_test,y_pred,pos_label = 'normal'),3))
print('f1_score ',round(nf1score ,3))

mean_f1 = ((af1score + nf1score)/2)
print('\nle f1-score moyen est de ',round(mean_f1,3) )

les meilleur parametre pour notre knn sont :  {'metric': 'manhattan', 'n_neighbors': 3}

pour la class ANOMALY :
precision =  0.919672131147541
recall =  0.9554357082032359
f1_score = 

 0.9372128637059725
pour la class NORMAL :
precision= 0.9597229348383787
recall= 0.9271375464684015
f1_score  0.9431488718013362

le f1 score moyen est de  0.9401808677536543
.score  0.9403281291346918


### Réponse de la dernière question :
#### Oui ce changement a un impact sur la performance du classificateur, on remarque que le f1-score du modèle de la phase 2 est plus bas que celui de la phase 1 ,  justification  le score obtenue de puis la phase 1 est le score pour la phase 2 qui se trouve juste au-dessus de cette cellule est qui est de  0.940
#### Remarque : il faut savoir que le score obtenue dans la phase 1 est un résultat biaiser à cause du  data-set qui n'ait pas homogène pour les deux class avec , 98,9 normal %, 1.1 anomaly %, le score de 1 ne reflète pas la performance de notre model sur des donner que il n’a jamais vu .
